In [ ]:
import requests
import os
import time
from bs4 import BeautifulSoup

In [28]:
def send_notification(alert, ntfy_topic):
    """Sends a notification using ntfy."""
    if not ntfy_topic:
        print("NTFY_TOPIC environment variable not set. Skipping notification.")
        return
        
    try:
        requests.post(
            f"https://ntfy.sh/{ntfy_topic}",
            data=alert['message'].encode('utf-8'),
            headers={
                "Title": f"New P2000 Alert: {alert['service']}",
                "Priority": "high",
                "Tags": "warning"
            })
        print("--> Notification sent!")
    except Exception as e:
        print(f"--> Failed to send notification: {e}")

In [ ]:
def clear_screen():
    """Clears the console screen."""    
    if os.name == 'nt':
        os.system('cls')
    else:
        os.system('clear')

In [ ]:
def get_region_from_env():
    """Gets region details from an environment variable."""
    # This is the full regions dictionary to look up the details
    regions = {
        "rb=1": "KNRM", "ll=1": "Lifeliners", "groningen=1": "Groningen",
        "friesland=1": "Friesland", "drenthe=1": "Drenthe", "ijsselland=1": "IJsselland",
        "twente=1": "Twente", "nogland=1": "Noord- en Oost Gelderland",
        "glandmidden=1": "Gelderland-Midden", "glandzuid=1": "Gelderland-Zuid",
        "utrecht=1": "Utrecht", "nholn=1": "Noord-Holland Noord",
        "zwland=1": "Zaanstreek-Waterland", "kennemerland=1": "Kennemerland",
        "adamal=1": "Amsterdam-Amstelland", "gooienvechtstr=1": "Gooi en Vechtstreek",
        "haagl=1": "Haaglanden", "holmid=1": "Hollands Midden",
        "rdamrm=1": "Rotterdam-Rijnmond", "zhz=1": "Zuid-Holland Zuid",
        "zeeland=1": "Zeeland", "mewbranant=1": "Midden- en West Brabant",
        "brabantn=1": "Brabant Noord", "brabantzo=1": "Brabant Zuid-Oost",
        "limburgn=1": "Limburg Noord", "limburgz=1": "Limburg Zuid", "flevol=1": "Flevoland",
        "": "Alle Regio's"
    }
    region_param = os.environ.get('P2000_REGION')
    if region_param in regions:
        return region_param, regions[region_param]
    return None, None # Return None if env var is not set or invalid

In [ ]:
def scrape(url):
    """Scrapes and returns the single latest alert from the given URL."""
    try:
        response = requests.get(url)
        response.raise_for_status()
        response.encoding = 'windows-1252'
        soup = BeautifulSoup(response.text, 'html.parser')

        all_alerts_raw = []
        main_alert_rows = soup.find_all(lambda tag: tag.name == 'tr' and tag.find('td', class_='DT'))

        for row in main_alert_rows:
            dt_cell = row.find('td', class_='DT')
            service_cell = row.find('td', class_=['Am', 'Br', 'Po'])
            region_cell = row.find('td', class_='Regio')
            message_cell = row.find('td', class_=['Md', 'Mdx'])

            if dt_cell and service_cell and region_cell and message_cell:
                all_alerts_raw.append({
                    "datetime": dt_cell.text.strip(),
                    "service": service_cell.text.strip(),
                    "region": region_cell.text.strip(),
                    "message": message_cell.text.strip(),
                })

        unique_alerts = []
        seen_alerts = set()
        for alert in all_alerts_raw:
            identifier = (alert['datetime'], alert['message'])
            if identifier not in seen_alerts:
                unique_alerts.append(alert)
                seen_alerts.add(identifier)

        return unique_alerts[0] if unique_alerts else None

    except requests.exceptions.RequestException as e:
        print(f"\nAn error occurred while trying to fetch the website: {e}")
        return None

--- No New Alert ---
Time:    24-06-2025 16:31:57
Service: Brandweer
Region:  Haaglanden
Message: P 2 BDH-03 Verontr. opp.water Overburgkade Voorburg 155630 159591 159020
--------------------


KeyboardInterrupt: Script stopped by user. Exiting.

In [ ]:
def main():
    """Main function to select a region and enter the automatic refresh loop."""
    clear_screen()
    
    region_param, region_name = get_region_from_env()

    ntfy_topic = os.environ.get('NTFY_TOPIC')
        
    base_url = 'http://www.p2000-online.net/p2000.py'
    url = f"{base_url}?{region_param}" if region_param else base_url
    
    print(f"--- Starting P2000 Alerter for region: {region_name} ---")
    if ntfy_topic:
        print(f"--- Notifications will be sent to ntfy.sh/{ntfy_topic} ---")
    else:
        print("--- Notifications are disabled (NTFY_TOPIC not set) ---")

    last_alert_identifier = None

    while True:
        latest_alert = scrape(url)

        if latest_alert:
            current_alert_identifier = (latest_alert['datetime'], latest_alert['message'])
            
            if current_alert_identifier != last_alert_identifier:
                clear_screen()
                print(f"--- New Alert for {region_name} ---")
                last_alert_identifier = current_alert_identifier
                
                print(f"Time:    {latest_alert['datetime']}")
                print(f"Service: {latest_alert['service']}")
                print(f"Region:  {latest_alert['region']}")
                print(f"Message: {latest_alert['message']}")
                print("--------------------")
                
                if "ZOETMR" in latest_alert['service'].upper():
                        print("--> Service matches 'ZOETMR', attempting to send notification...")
                        send_notification(latest_alert, ntfy_topic)
                else:
                    print("--> Service does not match 'ZOETMR', skipping notification.")
        
        time.sleep(10)

In [ ]:
if __name__ == "__main__":
    main()